In [1]:
# import the appropriate libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn.objects as so
from scipy import stats
from collections import Counter
import re
from datetime import datetime
import json
from SPARQLWrapper import SPARQLWrapper, JSON
import requests
from io import BytesIO
import ast

#image analysis: first install dlib in your ada environment (1: conda activate ada , 2: install dlib)
import dlib 
#face recognition library, first install it in your ada environment (1: conda activate ada, 2: install face_recognition)
import face_recognition
from PIL import Image, ImageDraw

ModuleNotFoundError: No module named 'dlib'

# Abstract

Have you ever wondered why some actors keep getting cast in the same role? Why Dwayne Johnson always seems to plays the adventurous, headstrong and caring leader? Why Jason Staham is constantly cast as the gritty, hardboiled tough guy? And do these recurring casting choices lead to box office success? <br>
This phenomenon is called typecasting, in which an actor becomes strongly identified with particular roles, or characters having the same traits or coming from the same social or ethnic groups. The CMU Movies Summary Corpus has already observed 501 recurring character tropes. In this study, we want to analyse the physical aspects of these tropes, by decoding the features *(gender, age, ethnicity and facial features)* of the actors observing them throughout time and movie genres, before correlating these findings to the movie's box office success. <br>

### Project Structure

### Part 1: Exploring and pre-processing the data
We identify the datasets of interest: *movies.metadata.tsv*, *characters.metadata.tsv* and *tvtropes.clusters.txt*.
Let us first identify the variables of interest for our story. <br>
- *Movie metadata: Wiki_movieID, Movie Name, Release Date, Box Office, Genres* <br>
- *Character metadata: Wiki_movieID, Release Date, Character Name, Gender, Height, Ethinicity, Actor Name, Age at Movie Release* <br>

**Step 1: Movie and Characters MetaData** <br>
    - We plot our variables of interest to visualise distributions, range and/or aberrant outliers. <br>
    - We analyse the percentage of missing data, giving us insight on how to complete them . <br>
    - We merge movies.metadata.tsv with characters.metadata.tsv to associate the characters with their respective genres, movie box office revenue and decade of release. <br>
    
**Step 2: TV Tropes** <br>
    - We pre-process our tvtropes.clusters.txt file and merge it with our merged character & movies metadata. We identify 501 character tropes with 72 unique tropes. <br>
    - We complete key missing data such as Actor Age which we calculate using DOB and Release Date <br>
    - Using WikiData, we convert the ethnicity labels to readable strings. <br>
    - We filter it by grouping tropes and ethnicities together before visualizing it. <br>
    - We do univariate and bivariate exploratory data analysis for our variables of interest. <br>
    
**Step 3: Extracting Actor Images** <br>
    - We extract the 350 actor images from [The Movie Database](https://www.themoviedb.org/) to create the Actor_image.csv dataset. <br>
    - We generated a personal API to extract the images URL and store them in a csv file with corresponding actor names.  We will use Dlib, a widely used pre-trained facial landmark detector to identify 68 key points on the face, storing them as pairs of coordinates. They will in turn used to derive additional features that we will link to the actors for our study. [Dlib annotations emphasize the importance of resolution and head pose accuracy in landmark placement. Illumination, quality, and color have a relatively minor impact](https://essay.utwente.nl/86867/1/Meijerink_BA_EEMCS.pdf). <br>
    <p align = "center">
        <img src = "https://github.com/epfl-ada/ada-2023-project-adacadabra1000101/assets/145772112/a8839649-0450-4d0d-9b08-e1fdbca23b2e.png" width = "400" height = "400"> <br>
    </p>
    - Despite having high-quality images and a well-trained model, we still encounter head pose challenges. In the upcoming steps, we will explore methods to rotate a 3D face to a pose comparable to a standard face. From these standardized images, we will extract facial landmarks to enhance the performance of the chosen model. <br>
    
### Part 2: Facial Image Analysis
**Step 4: Extracting Facial Features** <br>
    - We use the extracted coordinates to derive additional facial features of our actor images.<br>
    - Our continuous features are calculated from the landmarks. We will address proportional considerations when comparing actor images, as length between landmarks does not provide information without a reference length. <br>
    - Proportionality also has the side benefit of normalizing our values. <br>

**Step 5: Analysing and Selecting Facial Features** <br>
    - We analyse and select the facial features that are most pertinent to our study, by correlating them with our tropes-character-movies data. <br>
    - We re-extract more features once the head-pose accuracy issue has been fixed. <br>

### Part 3: Classifying our data
**Step 6: Putting all the features together** <br>
    - We put all the features, obtained from tropes-character-movies and the facial image analysis. <br>
    - We filter the features, keeping only the most important. <br>
    - We select a model and tune its parameters <br>

### Part 4: Data Analysis
**Step 7: Analysis** <br>
    - We analyse our data and find answers to our research questions. <br>

### Part 5: Story-Telling
**Step 8: Web Design** <br>
    - We use our findings to create our story on the webpage. <br>

### Part 6: When the fun *finally* begins
**Step 9: Adding our own images** <br>
    - We add our own images and use what we found earlier to determine what character trope would suit each of us best. <br>

***BONUS step: Adding an interactive feature*** <br>
    - We add a feature where a user can upload his own image and/or fill a form. We would then analyse his image and/or form to show the character trope that suits the user best.

#### Ethical Risks of our study
+ CMU Movie Summaries Corpus regroups a mix of nationalities and ethnicities for ActorEthnicity *(for example: Japanese Brasilians)*. We decided to cluster similar ActorEthnicity values together. This runs the risk of misrepresentation as we will potentially different ethnic groups together.
+ We use and analyze actor images from [The Movie Database](https://www.themoviedb.org/) (TMDB). As such, we are subject to the conditions detailed in TMDB's [Terms of Use](https://www.themoviedb.org/terms-of-use), notably in terms of the actors' image consent.

## Step 1: Movie and Characters MetaData

### a) Loading Data and Identifying variables of interest

In [ ]:
#Loading our data
characters_df = pd.read_csv('data/character.metadata.tsv', delimiter="\t", 
                            names=['Wiki_movieID', 'Freebase_movieID', 'ReleaseDate', 'CharacterName', 'ActorDOB', 'ActorGender', 'ActorHeight', 'ActorEthnicity', 'ActorName', 'ActorAge', 'Freebase_charactermap', 'Freebase_characterID', 'Freebase_actorID'])
movies_df = pd.read_csv('data/movie.metadata.tsv', delimiter="\t", 
                        names=['Wiki_movieID', 'Freebase_movieID', 'MovieName', 'ReleaseDate', 'BoxOffice', 'Runtime', 'Languages', 'Countries', 'Genres'])

#We look at the length and shape of our initial dataframes.
print("Size of 'Characters' DataFrame: %d" % len(characters_df))
print("Size of 'Movies' DataFrame: %d" % len(movies_df))

print("Shape of 'Characters' DataFrame: %d" % characters_df.shape[1])
print("Shape of 'Movies' DataFrame: %d" % movies_df.shape[1])

In [ ]:
#display column names and first line of characters dataframe
characters_df.head(1)

In [ ]:
#display column names and first line of movies dataframe
movies_df.head(1)

We identify the variables of interest for our project.
+ Movie metadata: *Wiki_movieID, MovieName, ReleaseDate, Genres, BoxOffice* <br>
+ Character metadata: *Wiki_movieID, ReleaseDate, CharacterName, ActorGender, ActorHeight, ActorEthinicity, ActorName, ActorAge* <br>

### b) Cleaning and Sorting our Data

In [ ]:
#Functions used for cleaning and sorting of data

def percentage_missingdata(dataframe, column_name):
# get the percentage of missing data from a column of a dataframe
# Args: dataframe: dataframe of the column, column_name
# Returns: percentage_missing
    notmissing = dataframe[column_name].count()
    totaldata = len(dataframe[column_name])
    percentage_missing = (1 - (notmissing/totaldata)) * 100
    
    return percentage_missing

def is_valid_date(date_string, format_string):
# is the date given valid in the given format
# Args: date_string : date to test, format_string: expected format
# Returns: boolean
    try:
        datetime.strptime(date_string, format_string)
        return True
    except ValueError:
        return False

In [ ]:
#change the {} (empty dict) into NaNs
movies_df.replace({'{}': pd.NA}, inplace=True)
characters_df.replace({'{}': pd.NA}, inplace=True)

#### b.i) Sorting and Visualising our Movie Dataframe

##### Decades of Release

In [ ]:
#We start by classifying our data per decades of release dates.
#Keeping only the year of release of the movies to classify them by decades
format1 = "%Y-%m-%d"
format2 = "%Y-%m"

for y in range(len(movies_df)):
    if isinstance(movies_df['ReleaseDate'][y], str):
        if is_valid_date(movies_df['ReleaseDate'][y],format1):
            date_obj = datetime.strptime(movies_df['ReleaseDate'][y], format1)
            movies_df.loc[y,'ReleaseDate'] = date_obj.year
            
    if isinstance(movies_df['ReleaseDate'][y], str):
        if is_valid_date(movies_df['ReleaseDate'][y],format2):
            date_obj = datetime.strptime(movies_df['ReleaseDate'][y], format2)
            movies_df.loc[y,'ReleaseDate'] = date_obj.year
            
movies_df['ReleaseDate'] = movies_df['ReleaseDate'].astype(float)
movies_df.sort_values(by='ReleaseDate', ascending=True, inplace=True)

#Visualization
display(movies_df[['MovieName', 'ReleaseDate', 'Genres', 'BoxOffice']].head(3))

>We notice an error in the movies dataset. The movie 'Hunting Season' is said to have a release date in 1010. This is impossible as the first movie was made in 1888, named 'Roundhay Garden Scene'. <br>
>A quick google search reveals that the movie's actual release year was 2010. Let us fix that.

In [ ]:
huntingseason_error = movies_df.loc[movies_df['MovieName'] == 'Hunting Season'].index
movies_df.loc[huntingseason_error,'ReleaseDate'] = 2010.0

#display after correction
display(movies_df[['MovieName', 'ReleaseDate', 'Genres', 'BoxOffice']].head(4))

> We notice two movies released in 1888 and 1889. To not have just two values in the 1880s decade, we decide to add these two movies to the 1890s.

In [ ]:
#This function returns the decade of the movie's release date.
def get_decade(year):
    return (year // 10) * 10

#Let us start by adding the decades to movies_df
if 'Decade' not in movies_df.columns:
    movies_df.insert(5, 'Decade', None)

movies_df['Decade'] = movies_df['ReleaseDate'].apply(lambda x: get_decade(x))

#As said earlier, we add the two movies released in 1888 and 1889 to the 1890s decades. 
decade_change1 = movies_df.loc[movies_df['ReleaseDate'] == 1888].index
movies_df.loc[decade_change1,'Decade'] = 1890.0

decade_change2 = movies_df.loc[movies_df['ReleaseDate'] == 1889].index
movies_df.loc[decade_change2,'Decade'] = 1890.0

# diplay these firest years and the decade they were assigned to
movies_df.sort_values(by='ReleaseDate' , ascending=True, inplace=True)
display(movies_df[['MovieName', 'Genres', 'Decade', 'ReleaseDate']].head(3))

In [ ]:
# simple plot of number of movies released per decade
plt.figure(figsize=(10, 4))
sns.countplot(x='Decade', data=movies_df, palette='viridis')
plt.title('Number of Movies Released per Decade')
plt.xlabel('Decade')
plt.ylabel('Number of Movies')
plt.show()

> We observe a gradual increase in number of movies until the 1980s *(with a drop in the 1940s due to WWII)* with a sudden burst in movie quantity in the 1990s and 2000s. <br>
> There is a small sample size for the 2010s because the data collected until 2016 only.

##### Movie Genres

In [ ]:
#Keeping only the name of the genre in our dataframe, instead of a dict with the freebase ID of the genre

genre_names_list = []

for gen in range(len(movies_df)):
    if pd.notna(movies_df['Genres'][gen]) and isinstance(movies_df['Genres'][gen], str):
        genre_dict = json.loads(movies_df['Genres'][gen])
        genre_names = list(genre_dict.values())
        genre_names_list.append(genre_names)
    else:
        genre_names_list.append(pd.NA)

movies_df.sort_index(ascending=True, inplace=True)
movies_df['Genres'] = genre_names_list

In [ ]:
#Flattening all the genres list into one single list to be able to count their occurencies
copy = movies_df.copy()
movies_df_exploded = copy.explode('Genres')
movies_df_exploded = movies_df_exploded[movies_df_exploded['Genres'] != '{}']
flattened_list = movies_df_exploded['Genres'].tolist()

genres_counts = Counter(flattened_list)
counts_df = pd.DataFrame.from_dict(genres_counts, orient='index', columns=['Count'])

In [ ]:
#Plotting movies by genres

#We notice that there's 2294 counts of films with missing genres, characterised by an NaN value. 
#As such we determine our threshold to be over 2300 for the genre occurences.
counts_df = counts_df[counts_df['Count'] >= 2300]

counts_df.sort_values(by='Count', ascending=True, inplace=True)

plt.figure(figsize=(10, 4))
sns.barplot(x=counts_df.index.tolist(), y=counts_df['Count'])
sns.regplot(x=np.arange(len(counts_df)), y=counts_df['Count'], order=7, scatter=False, color='red')
plt.xticks(rotation=90)
plt.ylim(bottom=0, top=35000)
plt.xlabel('Most Prevalent Genres')
plt.ylabel('Number of Movies')
plt.title('Number of Movies per Genre')
plt.tight_layout()
plt.show()

> We observe the 26 most prevalent movie genres in our database *(with more than 2400 movies per genre)* <br>
> A majority of the movies are dramas.

#### b.ii) Checking the percentage of missing data for each variable of interest

In [ ]:
#Missing data in movies_df
movie_variables = ['Wiki_movieID', 'MovieName', 'ReleaseDate','Genres', 'BoxOffice']
percentage_missing_movies = []

percentage_missing_movies = [percentage_missingdata(movies_df, var) for var in movie_variables]

missing_data_movies_df = pd.DataFrame({'Variables of Interest': movie_variables,
                                           'Percentage of Missing Data': percentage_missing_movies})  


In [ ]:
#Plotting the percentage of missing data for the variables of movies_df
plt.figure(figsize=(6, 3))
bars = plt.bar(missing_data_movies_df['Variables of Interest'], missing_data_movies_df['Percentage of Missing Data'])
plt.ylim(0, 100)
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, yval, round(yval, 1), ha='center', va='bottom')

plt.xlabel('Variables of Interest')
plt.ylabel('Missing Data (in %)')
plt.title('Percentage of Missing Data per Variable in the Movies DataFrame')
plt.tight_layout()
plt.show()

>We observe 0% of missing data for Wiki_movieID and Movie Name. We can associate each movie to its respective Wikipedia Page using WikiData without any errors.<br>
>We have 8.4% of missing data for Release Date variable and 2.8% for Genres variable. These values are both very low and under 10%. Generally, when we have more than 10% of missing data, our data is likely to be biased. As such this is not the case for these two variables.<br>
>We see however 89.7% of missing data for the BoxOffice variable, an astonishing amount. This will mean that our BoxOffice is very likely to be biased towards the movies that have a BoxOffice value.<br>

In [ ]:
#Missing data in characters_df
character_variables = ['Wiki_movieID', 'ReleaseDate', 'CharacterName','ActorGender', 'ActorHeight', 
                                                    'ActorEthnicity', 'ActorName', 'ActorAge','ActorDOB']
percentage_missing_characters = []

percentage_missing_characters = [percentage_missingdata(characters_df, var) for var in character_variables]

missing_data_characters_df = pd.DataFrame({'Variables of Interest': character_variables,
                                           'Percentage of Missing Data': percentage_missing_characters})                                                                                                                             

In [ ]:
#Plotting the percentage of missing data for the variables of characters_df
plt.figure(figsize=(6, 4))
bars = plt.bar(missing_data_characters_df['Variables of Interest'], missing_data_characters_df['Percentage of Missing Data'])
plt.ylim(0, 100)
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, yval, round(yval, 1), ha='center', va='bottom')

plt.xticks(rotation=45)
plt.xlabel('Variables of Interest')
plt.ylabel('Missing Data (in %)')
plt.title('Percentage of Missing Data per Variable in the Characters DataFrame')
plt.tight_layout()
plt.show()

>We observe a high percentage of missing data for our respective variables of interest, the notable ones being *CharacterName*, *ActorHeight*, *ActorEthnicity* and *ActorDOB*, *ActorAge*.<br>
> For character name, this missing data is to be expected as many characters in movies do not have names, being background characters. Their character roles are much more interesting for our analysis. <br>
> Similarly, the missing data in actor age and actor date of birth (DOB) is not an issue as we can use DOB and Release Date to calculate the actor's age. The inverse is true to find the DOB. We can clean these missing values. <br>
> Our biggest concern is with the large quantity of missing data in both actor height and ethnicity as they are both key features to our project idea. We can use WikiData to complete our data. <br>

### c) Merging Movie and Character DataFrames


In [ ]:
characters_movies_df = movies_df.merge(characters_df, on=['Wiki_movieID'], how = 'left', suffixes=("", "_y")).drop(['Freebase_movieID_y', 'ReleaseDate_y'], axis=1)

#Visualization
display(characters_movies_df[['CharacterName', 'ActorName', 'ActorAge', 'ActorDOB', 'ActorEthnicity',
                                  'ActorGender','ActorHeight', 'MovieName', 'Genres', 'Decade', 'ReleaseDate', 'BoxOffice']].head(3))

In [ ]:
#Counting the number of characters per movie genre
characters_movies_copy = characters_movies_df.copy()
c_m_exploded = characters_movies_copy.explode('Genres')
c_m_exploded = c_m_exploded[c_m_exploded['Genres'] != '{}']
flat_list = c_m_exploded['Genres'].tolist()

charac_counts = Counter(flat_list)
charac_counts_df = pd.DataFrame.from_dict(charac_counts, orient='index', columns=['Count'])

In [ ]:
charac_counts_df = charac_counts_df[charac_counts_df['Count'] >= 15000]

charac_counts_df.sort_values(by='Count', ascending=True, inplace=True)

plt.figure(figsize=(10, 4))
sns.barplot(x=charac_counts_df.index.tolist(), y=charac_counts_df['Count'])
sns.regplot(x=np.arange(len(charac_counts_df)), y=charac_counts_df['Count'], order=7, scatter=False, color='red')
plt.xticks(rotation=90)
plt.ylim(bottom=0, top=200000)
plt.xlabel('Most Prevalent Genres')
plt.ylabel('Number of Characters')
plt.title('Number of Characters per Genre')
plt.tight_layout()
plt.show()

>We plot the number of characters per genre. The distribution matches the earlier plotted distribution of number of movies per genre with a majority of characters being in dramas or comedies.

## Step 2: TV Tropes

### a) Visualising our TV Tropes Dataframe

In [ ]:
df_tropes_original = pd.read_csv('data/tvtropes.clusters.txt', delimiter="\t", names=['trope', 'character'])
json_obj = df_tropes_original['character'].apply(json.loads)
df_tropes_norm = pd.json_normalize(json_obj[0])

for i in range(1, len(json_obj)):
    df_norm = pd.json_normalize(json_obj[i])
    df_tropes_norm = pd.concat([df_tropes_norm, df_norm], ignore_index=True)

tropes_df = pd.concat([df_tropes_original.drop('character', axis=1), df_tropes_norm], axis=1)

tropes_df = tropes_df.rename(columns={'id': 'Freebase_charactermap', 'char':'CharacterName', 'actor':'ActorName', 'movie':'MovieName'})

#Visualizing tropes_df data
display(tropes_df.head())
print("Size of 'Tropes' DataFrame: %d" % len(tropes_df))
print("Shape of 'Tropes' DataFrame: %d" % tropes_df.shape[1])

>We visualise our tvtropes.clusters as a dataframe and we observe 501 character tropes with 72 unique types. We will now merge it with our characters_movies_df to associate the characters to their respective tropes. <br> 
>We will then check for missing data in this new merged dataframe. <br>

### b) Merging Character & Movies Dataframe on TV Tropes

In [ ]:
# population de tropes_df with the df containing the movies and characters
tropes_movie_character_df= pd.merge(tropes_df, characters_movies_df, on=['CharacterName', 'ActorName', 'MovieName', 'Freebase_charactermap'], how='left').drop(['Runtime', 'Languages', 'Countries','Freebase_characterID'], axis=1)
new_order=['Freebase_charactermap', 'trope', 'CharacterName', 'ActorName', 'ActorAge','ActorDOB', 'ActorEthnicity', 'ActorGender', 'ActorHeight', 'Freebase_actorID', 'MovieName', 'Genres','Decade', 'ReleaseDate', 'BoxOffice', 'Freebase_movieID','Wiki_movieID' ]
tropes_movie_character_df=tropes_movie_character_df[new_order]
tropes_movie_character_df=tropes_movie_character_df.rename(columns={'trope': 'Trope'})
tropes_movie_character_df=tropes_movie_character_df.sort_values('Trope')

#Visualiszing our merged dataframe, tropes_movie_character_df:
display(tropes_movie_character_df[['Trope', 'CharacterName', 'ActorName', 'ActorAge', 'ActorDOB', 'ActorEthnicity',
                                  'ActorGender','ActorHeight', 'MovieName', 'Genres', 'Decade', 'ReleaseDate', 'BoxOffice']]
                                  .head(3))
print("Size of 'Tropes-Movie-Character' DataFrame: %d" % len(tropes_movie_character_df))
print("Shape of 'Tropes-Movie-Character' DataFrame: %d" % tropes_movie_character_df.shape[1])

>The size of our dataframe matches the size of our Tropes DataFrame indicating a correct merge. We have associated the 501 character tropes to their respective characters, movies and features *(Actor Name, Movie Genre, Actor Ethnicity...)*.

### c) Visualising the missing data in TV Tropes

In [ ]:
#Missing data for tropes
tropes_variables = ['Wiki_movieID', 'MovieName', 'ReleaseDate','Genres', 'BoxOffice', 'CharacterName','ActorGender', 'ActorHeight', 
                                                    'ActorEthnicity', 'ActorName', 'ActorAge','ActorDOB']
percentage_missing_tropes_data = []

percentage_missing_tropes_data = [percentage_missingdata(tropes_movie_character_df, var) for var in tropes_variables]
    
missing_data_tropes_df = pd.DataFrame({'Variables of Interest': tropes_variables,
                                           'Percentage of Missing Data': percentage_missing_tropes_data})

In [ ]:
#Plotting the percentage of missing data for the variables of missing_data_tropes_df
plt.figure(figsize=(5, 4))
bars = plt.bar(missing_data_tropes_df['Variables of Interest'], missing_data_tropes_df['Percentage of Missing Data'])
plt.ylim(0, 100)
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, yval, round(yval, 1), ha='center', va='bottom')

plt.xlabel('Variables of Interest')
plt.ylabel('Percentage of Missing Data (in %)')
plt.title('Percentage of Missing Data per Variable in the Movies DataFrame')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

>We notice a dramatic decrease in missing data compared to our movies_df and characters_df dataframes, with relatively small amounts for all the variables of interests except *ActorEthnicity*. <br>
>We will now clean our data, starting with the 3.6% of missing data in Actor Age that we can easily fix using ActorDOB. <br>

#### c.i) Filling the missing data in Actor Age

In [ ]:
#Finding the entries missing both ActorAge and ActorDOB
missing_age=tropes_movie_character_df[tropes_movie_character_df['ActorAge'].isna() & tropes_movie_character_df['ActorDOB'].isna()]

#Display the entries missing both ActorAge and ActorDOB
display(missing_age[['Trope', 'CharacterName', 'ActorName', 'ActorAge', 'ActorDOB', 'ReleaseDate']])

> As before, we want to fill the 3.6% of missing data in Actor Age by calculating it with the date of birth of the actor and the year of release. 
> As noted in the table above, only two actors do not have either Actor Age or Date of Birth, as such we might have to manually fill in the data for these two actors.

In [ ]:
#Converting DOB to Years
for y in range(len(tropes_movie_character_df)):
    if isinstance(tropes_movie_character_df['ActorDOB'][y], str):
        if is_valid_date(tropes_movie_character_df['ActorDOB'][y],format1):
            date_obj = datetime.strptime(tropes_movie_character_df['ActorDOB'][y], format1)
            tropes_movie_character_df.loc[y,'ActorDOB'] = date_obj.year

In [ ]:
#Calculating Actor Ages and filling the missing data
tropes_movie_character_df['ActorDOB'] = tropes_movie_character_df['ActorDOB'].astype(float)

tropes_movie_character_df['ActorAge'] = tropes_movie_character_df['ReleaseDate'] - tropes_movie_character_df['ActorDOB']

In [ ]:
#Checking missing data in ActorAge again
age_variables = ['ActorAge','ActorDOB']
percentage_missing_tropes_age = []

percentage_missing_tropes_age = [percentage_missingdata(tropes_movie_character_df, var) for var in age_variables]
  
missing_data_tropes_age = pd.DataFrame({'Variables of Interest': age_variables,
                                           'Percentage of Missing Data': percentage_missing_tropes_age})

print(missing_data_tropes_age)

>As seen in the table above, we have successfully completed the missing data in Actor Age, with the percentage decreasing from 3.6% to 0.399% *(the percentage of missing data for ActorDOB)*. <br>
>With a quick calculation, we observe that the remaining 0.399% of missing data in Actor Age and Actor Date of Birth corresponds to the two actors we identified earlier who do not have values for both ActorAge and ActorDOB. Let us manually add these values to our data frame. <br>
<br>
> Our missing actors and characters are: <br> - *Prince Otwani played by Trevor Thomas in the movie 'Sheena'*  <br> - *Prince Yu played by Junjie Qin in the movie 'Curse of the Golden Flower'* <br>
> Upon printing the rows with missing data for both ActorAge and ActorDOB above, we identified the indexes for both missing values: index 233 for Trevor Thomas and 227 for Junjie Qin.

In [ ]:
tropes_movie_character_df.loc[227,'ActorDOB'] = 1991.0
tropes_movie_character_df.loc[227, 'ActorAge'] = tropes_movie_character_df['ReleaseDate'][227] - tropes_movie_character_df['ActorDOB'][227] 

> Despite our best efforts, we could not find any information on Trevor Thomas on his date of birth or age. As such, we decide to leave him as a missing value for our dataframe.

In [ ]:
#We print the percentage of missing data for tropes_movie_character_df one last time.
percentage_missing_tropes_age_new = []

percentage_missing_tropes_age_new = [percentage_missingdata(tropes_movie_character_df, var) for var in age_variables]
    
missing_data_tropes_age_new = pd.DataFrame({'Variables of Interest': age_variables,
                                           'Percentage of Missing Data': percentage_missing_tropes_age_new})

print(missing_data_tropes_age_new)

> We now have less than 0.2% of missing data for both ActorDOB and ActorAge.

In [ ]:
missing_ethn=tropes_movie_character_df[tropes_movie_character_df['ActorEthnicity'].isna()]

print('The number of actors with missing ActorEthnicity values: %d' %len(missing_ethn['ActorName']))
print('This matches the 33.5% of missing data calculated above.')


#### d) Transforming the ActorEthnicity label in a string using [WikiData](https://query.wikidata.org/sparql)

> We observe that the ActorEthnicity is charactertised by a label, for example */m/09v5bdn*. Let us use WikiData to retrieve these labels and display them in a readable fashion.

In [ ]:
def get_ethnicity(freebase_id):
    # get the label of the freebase ID from Wikidata
    # Args: freebase_id: the freebase ID to be decoded
    # Returns: label, the corresponding label of the Wikidata ID (and a specific text for when)
    
    endpoint_url = "https://query.wikidata.org/sparql"
    query = """
    SELECT ?item ?itemLabel WHERE {
      ?item wdt:P646 '""" + freebase_id + """'.
      SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
    }
    """
    sparql = SPARQLWrapper(endpoint_url)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    
    try:
        results = sparql.query().convert()
        if results["results"]["bindings"]:
            label = results["results"]["bindings"][0]["itemLabel"]["value"]
            return label
        else:
            return "Label not found for the specified Freebase ID"
    except Exception as e:
        return np.nan

In [ ]:
# groups by ethnicity, to get all the different ethnicities in one table
temp_df = tropes_movie_character_df['ActorEthnicity'].groupby(tropes_movie_character_df['ActorEthnicity']).agg(['count'])
temp_df = temp_df.reset_index()
temp_df.head(3)

> As seen above, we have all the ethnicities in one table labelled in a key. We will use WikiData match every freebase ethnicity ID to its corresponding word.

In [ ]:
#Match every freebase ethnicity ID to the ethnicity, if it found in Wikidata
temp_temp_df = temp_df.copy()
et = []
for i in range(len(temp_df['ActorEthnicity'])):
    df_temp_ethnicity = temp_df.iloc[i:i+1]
    for ethnicity in df_temp_ethnicity['ActorEthnicity'] :
        eth_tra = get_ethnicity(ethnicity)
        j = 0
        while ((type(eth_tra) == float) & (j<100)):
            #print(type(get_ethnicity(ethnicity)))
            eth_tra = get_ethnicity(ethnicity)
            j += 1
        et.append(eth_tra)

In [ ]:
temp_temp_df['StrActorEthnicity'] = et

In [ ]:
#convert to string every ethnicity taken from Wikidata
tmp = []
for i in range (len(temp_temp_df['StrActorEthnicity'])):
    if not (pd.isna(temp_temp_df['StrActorEthnicity'][i])):
        tmp.append(str(temp_temp_df['StrActorEthnicity'][i]))
    else:
        tmp.append(temp_temp_df['StrActorEthnicity'][i])
temp_temp_df['StrActorEthnicity'] = tmp
ethnicity_decoding = temp_temp_df

#len(ethnicity_decoding['StrActorEthnicity'])
ethnicity_decoding.head()

> We associated the Actor Ethnicity labels to their respective strings. We observe another issue though. <br>
> We have a very large number of ethnicities with very precise descriptions, for example *Asian American, Korean American...* We decide to simplify and group our ethnicities together. <br>
> We do this by deleting the word *"people"* and "*person*" after space, allowing for example *Irish* and *Irish people* to merge into the same category. We also assume the first ethnicity to be more representative then "*american"*. For example, "*Korean American*" will be labelled in our dataset as *Korean*. <br>
> We also transform all the uppercases into lowercases. <br>

In [ ]:
# simplifying the ethnicity_decoding dataframe. 
simple_ethnicity_decoding = ethnicity_decoding.copy()
simple_ethnicity_decoding['StrActorEthnicity'] = simple_ethnicity_decoding['StrActorEthnicity'].str.replace(' Americans', '')
simple_ethnicity_decoding['StrActorEthnicity'] = simple_ethnicity_decoding['StrActorEthnicity'].str.replace(' American', '')
simple_ethnicity_decoding['StrActorEthnicity'] = simple_ethnicity_decoding['StrActorEthnicity'].str.replace(' people', '')
simple_ethnicity_decoding['StrActorEthnicity'] = simple_ethnicity_decoding['StrActorEthnicity'].str.replace(' person', '')
simple_ethnicity_decoding['StrActorEthnicity'] = simple_ethnicity_decoding['StrActorEthnicity'].str.lower()
len(simple_ethnicity_decoding['StrActorEthnicity'].unique())
simple_ethnicity_decoding.head()

>We observe the results of ethnicity simplication above, with *Asian American* becoming *Asian* etc

In [ ]:
tropes_characters_ethnicity_df = tropes_movie_character_df.merge(simple_ethnicity_decoding, on='ActorEthnicity', how='left').drop(['count'], axis = 1)
tropes_characters_ethnicity_df.sort_values(by='CharacterName', ascending=True, inplace=True)
tropes_characters_ethnicity_df = tropes_characters_ethnicity_df.rename(columns={'ActorEthnicity': 'ActorEthnicity Label'})

#Visualizing our data
display(tropes_characters_ethnicity_df[['Trope', 'CharacterName', 'ActorName', 'ActorEthnicity Label', 'StrActorEthnicity']].head())

> As seen above, we successfully retrieved the actor ethnicities for the 67.5% of actors with ActorEthnicity labels.

In [ ]:
#finding the number of labels for which the function was unable to retrieve a label, the ethnicity is saved as 'Label not found for the specified Freebase ID'
missing_label_df = tropes_characters_ethnicity_df[tropes_characters_ethnicity_df['StrActorEthnicity'].str.contains('Label', case=False, na=False)]

#we replace the text 'Label not found for the specified Freebase ID' by np.Na as it is unusable data
tropes_characters_ethnicity_df['StrActorEthnicity'].replace({'label not found for the specified freebase id': pd.NA}, inplace=True)

In [ ]:
#Counting the number of characters per ethnicity
tropes_characters_ethnicity_copy = tropes_characters_ethnicity_df.copy().dropna()
flat_list_ethn = tropes_characters_ethnicity_copy['StrActorEthnicity'].tolist()

tropes_counts = Counter(flat_list_ethn)
tropes_counts_df = pd.DataFrame.from_dict(tropes_counts, orient='index', columns=['Count'])

In [ ]:
# plotting the number of characters per ethnicity using the counts from the previous cell
fig, ax = plt.subplots(figsize=(12, 7))

tropes_counts_df.sort_values(by='Count', ascending=True, inplace=True)

sns.barplot(x=tropes_counts_df.index.tolist(), y=tropes_counts_df['Count'])
sns.regplot(x=np.arange(len(tropes_counts_df)), y=tropes_counts_df['Count'], order=7, scatter=False, color='red', ax=ax)
plt.xticks(rotation=90)
plt.ylim(bottom=0, top=42)
plt.xlabel('Ethnicities')
plt.ylabel('Number of Characters')
plt.title('Number of Characters per Ethnicity')
plt.tight_layout()
plt.show()

### d) Plotting the data in our merged tropes_characters df

#### d.i) Univariate Exploratory Data Analysis (EDA)

In [ ]:
fig, axes = plt.subplots(nrows=4, ncols=2, figsize=(15, 10))
sns.histplot(x=tropes_characters_ethnicity_df['BoxOffice'], color = 'red', ax=axes[0,0], kde=True)
axes[0,0].set_title('Box Office Histogram')

sns.boxplot(x=tropes_characters_ethnicity_df['BoxOffice'], color = 'red', ax=axes[0,1])
axes[0,1].set_title('Box Office Box Plot')

sns.histplot(x=tropes_characters_ethnicity_df['ActorHeight'], color = 'lime', ax = axes[1,0], kde = True)
axes[1,0].set_title('Actor Height Histogram')

sns.boxplot(x=tropes_characters_ethnicity_df['ActorHeight'], color = 'lime', ax = axes[1,1])
axes[1,1].set_title('Actor Height Box Plot')

sns.histplot(x=tropes_characters_ethnicity_df['ActorAge'], color = 'pink', ax = axes[2,0], kde = True)
axes[2,0].set_title('Actor Age Histogram')

sns.boxplot(x=tropes_characters_ethnicity_df['ActorAge'], color = 'pink', ax = axes[2,1])
axes[2,1].set_title('Actor Age Box Plot')

sns.countplot(x='Decade', data=tropes_characters_ethnicity_df, palette='viridis', ax = axes[3,0])
axes[3,0].set_title('Number of Movies Released per Decade')
axes[3,0].set_xlabel('Decade')
axes[3,0].set_ylabel('Number of Movies')

sns.boxplot(x=tropes_characters_ethnicity_df['Decade'], palette = 'viridis', ax = axes[3,1])
axes[3,1].set_title('Movies per Decade Box Plot')

plt.tight_layout()
plt.show()

> + For BoxOffice, we observe a skewed power law distribution, with a large number of extreme outliers and a large concentration of movies having a low box office number (in terms of millions). <br>
> + For ActorHeight, we observe a bimodal distribution that could be explained by the presence of two populations, Female and Male actors. We will plot this distribution in function of gender to correlate with our observation. <br>
> + For Actor Age, we observe a normal distribution, with the mean being at around 37 years old. There are though a few outliers in the 70s and 80s. <br>
> + We plot the number of movies by decade. We notice a similar distribution to our initial plot of movies per decade, for movies_df. As such, our filter has not biased the decade distribution of our movies. Our boxplot shows the same skewed distribution towards the left, with a outliers in the 1920s to 1940s. <br>

#### d.ii) Bivariate EDA

##### Actor Height per Gender

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

sns.scatterplot(x='ActorName', y='ActorHeight', hue='ActorGender', data=tropes_characters_ethnicity_df, 
                palette={'M': 'pink', 'F': 'green'}, s=150, ax=axes[0])
axes[0].set_xticks([])
axes[0].set_xlabel('')
axes[0].set_ylabel('Height (in meters)')
axes[0].set_title('Actor Height Scatter Plot')
axes[0].legend(loc='upper left', bbox_to_anchor=(0, 1))

sns.histplot(x=tropes_characters_ethnicity_df['ActorHeight'], color = 'lime', ax = axes[1], 
             hue=tropes_characters_ethnicity_df['ActorGender'],palette={'M': 'pink', 'F': 'green'}, kde = True)
axes[1].set_title('Actor Height Histogram')
axes[1].set_xlabel('Actor Heights (in meters)')
axes[1].set_ylabel('')
axes[1].set_title('Actor Height Histogram')

plt.tight_layout()
plt.show()

>Our earlier hypothesis about ActorHeight distribution being a bimodal distribution sectioned by gender is wrong, as indicated in the histogram on the left. Indeed, the population of female actors is too low to impact the male actor height distribution.

##### Height by Trope

In [ ]:
sns.barplot(x='Trope', y = 'ActorHeight', data= tropes_characters_ethnicity_df)
plt.xticks(rotation=90)
plt.ylim(bottom=1.65, top = 1.9)
plt.xlabel('Character Tropes')
plt.ylabel('Actor Height (in meters)')
plt.title('Average Height per Character Trope')
plt.tight_layout()
plt.show()

> We plot the average height per character trope. We notice that some tropes have much lower average height. This could be due to some tropes be stereotypically feminine ones.

##### Number of Characters per Trope

In [ ]:
tropes_characters_ethnicity_df['Trope'].hist(bins=72, xrot=90,figsize=(15,6))
plt.title('Number of Character per Trope Histogram')
plt.xlabel('Trope')
plt.ylabel('Count')
plt.show()

>We plot the number of characters by tropes. We notice that some character tropes have very low numbers of characters associated, such as *classy_cat_burglar, klutz*, and, *pupil_turned_to_evil*. <br>
>A solution to this issue could be grouping similar character tropes together, for example *klutz* could be associated with *ditz* to constitute the trope *clumsy_person*.
>Sadly we do not have access to the FreebaseID Dataset that David Bamman, Brendan O'Connor, and Noah Smith used to create their unique character personas as it is not available anymore. They based themselves on key words from the plot summaries to create character maps and derive the tropes from said maps. <br>
>As such, we will opt for a more rudimentary approach by manually grouping similar tropes together. This will be done in function of their definition on [TV Tropes](https://tvtropes.org/), as used by Bamman, O'Connor & Smith and our personal knowledge.<br>

In [ ]:
#Let us create the archetypes detailed above as 15 different lists
skilled_badass = ['master_swordsman', 'warrior_poet', 'cultured_badass', 'fastest_gun_in_the_west', 
                  'adventurer_archaeologist', 'hitman_with_a_heart']
loser = ['doormat', 'coward', 'henpecked_husband', 'dean_bitterman', 'loser_protagonist']
laidback_freebird = ['junkie_prophet', 'slacker', 'stoner', 'surfer_dude']
jock = ['bully', 'jerk_jock', 'dumb_muscle', 'arrogant_kungfu_guy', 'bruiser_with_a_soft_center']
charismatic_charmer = ['loveable_rogue', 'byronic_hero', 'casanova', 'charmer', 'big_man_on_campus', 
                       'gentleman_thief', 'classy_cat_burglar']
respected_leader = ['officer_and_a_gentleman', 'father_to_his_men', 'self_made_man', 'the_chief']
crazy_fighter = ['bounty_hunter', 'psycho_for_hire', 'tranquil_fury', 'crazy_survivalist']
dumb_and_clumsy = ['brainless_beauty', 'dumb_blonde', 'ditz', 'klutz', 'stupid_crooks']
shallow_and_popular = ['final_girl', 'valley_girl', 'prima_donna', 'granola_person', 'chanteuse']
old_wise_quirky = ['eccentric_mentor', 'retired_outlaw', 'hardboiled_detective', 'absent_minded_professor']
sidekick = ['romantic_runnerup', 'storyteller', 'young_gun', 'bromantic_foil']
emotional_damage = ['heartbroken_badass', 'ophelia', 'broken_bird']
evil_character = ['revenge', 'evil_prince', 'egomaniac_hunter', 'pupil_turned_to_evil', 'trickster']
mean_officer = ['consummate_professional', 'dirty_cop', 'corrupt_corporate_executive', 'the_editor', 'grumpy_old_man', 
                'drill_sargeant_nasty', 'morally_bankrupt_banker']
tech_genius = ['child_prodigy', 'gadgeteer_genius', 'playful_hacker']

In [ ]:
#Replacing the tropes in tropes_character_ethnicity_df by their new grouped archetypes one by one
tropes_characters_ethnicity_df['Trope'] = tropes_characters_ethnicity_df['Trope'].replace(to_replace=skilled_badass, 
                                                                                          value='skilled_badass')
tropes_characters_ethnicity_df['Trope'] = tropes_characters_ethnicity_df['Trope'].replace(to_replace=loser, 
                                                                                          value='loser')
tropes_characters_ethnicity_df['Trope'] = tropes_characters_ethnicity_df['Trope'].replace(to_replace=laidback_freebird, 
                                                                                          value='laidback_freebird')
tropes_characters_ethnicity_df['Trope'] = tropes_characters_ethnicity_df['Trope'].replace(to_replace=jock, 
                                                                                          value='jock')
tropes_characters_ethnicity_df['Trope'] = tropes_characters_ethnicity_df['Trope'].replace(to_replace=charismatic_charmer, 
                                                                                          value='charismatic_charmer')
tropes_characters_ethnicity_df['Trope'] = tropes_characters_ethnicity_df['Trope'].replace(to_replace=respected_leader, 
                                                                                          value='respected_leader')
tropes_characters_ethnicity_df['Trope'] = tropes_characters_ethnicity_df['Trope'].replace(to_replace=crazy_fighter, 
                                                                                          value='crazy_fighter')
tropes_characters_ethnicity_df['Trope'] = tropes_characters_ethnicity_df['Trope'].replace(to_replace=dumb_and_clumsy, 
                                                                                          value='dumb_and_clumsy')
tropes_characters_ethnicity_df['Trope'] = tropes_characters_ethnicity_df['Trope'].replace(to_replace=shallow_and_popular, 
                                                                                          value='shallow_and_popular')
tropes_characters_ethnicity_df['Trope'] = tropes_characters_ethnicity_df['Trope'].replace(to_replace=old_wise_quirky, 
                                                                                          value='old_wise_quirky')
tropes_characters_ethnicity_df['Trope'] = tropes_characters_ethnicity_df['Trope'].replace(to_replace=sidekick, 
                                                                                          value='sidekick')
tropes_characters_ethnicity_df['Trope'] = tropes_characters_ethnicity_df['Trope'].replace(to_replace=emotional_damage, 
                                                                                          value='emotional_damage')
tropes_characters_ethnicity_df['Trope'] = tropes_characters_ethnicity_df['Trope'].replace(to_replace=evil_character, 
                                                                                          value='evil_character')
tropes_characters_ethnicity_df['Trope'] = tropes_characters_ethnicity_df['Trope'].replace(to_replace=mean_officer, 
                                                                                          value='mean_officer')
tropes_characters_ethnicity_df['Trope'] = tropes_characters_ethnicity_df['Trope'].replace(to_replace=tech_genius, 
                                                                                          value='tech_genius')

In [ ]:
plt.figure(figsize=(15, 5))
trope_plot = sns.histplot(data=tropes_characters_ethnicity_df, x='Trope', bins=16, edgecolor='black', stat='count', 
             hue='Trope', palette='deep',legend = False, kde=False)
plt.xticks(rotation=90)
plt.title('Distribution of Tropes')
plt.xlabel('Trope')
plt.ylabel('Count')
plt.tight_layout()

# Show the plot
plt.show()

##### Exploring the relations between genres and tropes

In [ ]:
#Duplicates row values for each individual genre found in Genres
tropes_genres_exploded=tropes_characters_ethnicity_df.explode('Genres')

In [ ]:
print('In our reduced dataset we count ' + str(tropes_genres_exploded['Genres'].nunique()) + ' unique genres')

>Having 182 unique genres, plotting the number of tropes appearances per genre would be a mess. Instead we can plot the number of unique genres each trope appears in. Such a plot would give us an insight wether or not a trope is specific for a certain genre or if tropes in general can be found in multiple genres.

In [ ]:
#Plots a histogram of the number of genres a given trope is present in
genre_count =pd.DataFrame(tropes_genres_exploded.groupby('Trope')['Genres'].nunique())
sns.barplot(data=genre_count,x=genre_count.index, y='Genres')
plt.xticks(rotation=90)
plt.title('Number of unique genres in which a trope appears ')
plt.show()

>From this plot we learn that our trope categories are general enought to cover at least 40 different genres. This will enables us to compare actors characteristics for a given trope accross different genres.

### Step 3: Actor Images Dataset

>In this part we want to extract actors' facial encodings, in order to use them as features for classification. We are using the *actor_images.csv* file that we've created. It contains actor names, their correspoding pictures stored as a url with it's width and height. The code for that file can be found in the *people_images.ipynb* notebook.   

In [ ]:
actor_images = pd.read_csv('data/our_datasets/actor_images.csv')
actor_images.head()

In [ ]:
#Finding missing values 
missing = actor_images == '-'
missing_images = actor_images[missing.any(axis=1)]
print(missing_images)

>Only 0.85% of the data is missing. We will disregard it for now as it is marginal

In [ ]:
#actor_images without the missing urls
actor_cleaned = actor_images[~missing.any(axis=1)]

In [ ]:
#get face encodings using the face_recognition library
#this cell runs for ~10 minutes
face_encodings = {'Actor': [], 'Encodings': []}

for index, row in actor_cleaned.iterrows():
    actor_name = row['Actor']
    image_url = row['Image URL']

    # Remove spaces from actor name
    #retrieve image url
    img = requests.get(image_url)
    image_content = BytesIO(img.content)

    #transmfor the image into an array
    img = np.asarray(Image.open(image_content))
    
    #extract facial encodings (coordinates)
    face_landmarks_list = face_recognition.face_landmarks(img)

    face_encodings['Encodings'].append(face_landmarks_list)
    face_encodings['Actor'].append(actor_name)


In [ ]:
#Create a Dataframe containing the facial encodings for each actor
face_encodings = pd.DataFrame(face_encodings)

>Having extracted the facial encodings for each actor, we want to retrieve features that we will use for classification. For the sake of comparison between actors, we will use proportions of certain landmarks with respect to two reference measures: one for face width (distance betweem the temples) and one for the face length (distance between the top of the nose bridge and the lowest point of the chin). For exemple, we will not look at the length of a nose, but as its size in proportion to the reference length. <br>
>For this milestone, we are only focussing on three features: nose length and width and distance between eyes. More features will be extracted for the final analysis. 

In [ ]:
def face_proportions(row):
    try:
        landmarks = row['Encodings'][0]  # Extract the dictionary from the list
    except (json.JSONDecodeError, IndexError):
        # Handle cases where 'Encodings' is not a valid JSON or the list is empty
        return pd.Series({'Actor': row['Actor'],'Eye Distance': np.nan, 'Nose Length': np.nan, 'Nose Width': np.nan})

    
    chin_landmarks = landmarks['chin']
    nose_landmarks = landmarks['nose_bridge']
    nose_width_landmarks = landmarks['nose_tip']

    if not chin_landmarks or not nose_landmarks or not nose_width_landmarks:
        # Handle cases where expected landmarks are not available
        return pd.Series({'Actor': row['Actor'],'Eye Distance': np.nan,'Nose Length': np.nan, 'Nose Width': np.nan})

    x1 = chin_landmarks[0][0]
    x2 = chin_landmarks[-1][0]
    y1 = nose_landmarks[0][1]
    y2 = chin_landmarks[9][1]

    x_max = np.abs(x2 - x1)
    y_max = np.abs(y2 - y1)

    left_eye = landmarks.get('left_eye', [])
    right_eye = landmarks.get('right_eye', [])

    if not left_eye or not right_eye:
        eye_dist = np.nan
    else: 
        x_r = right_eye[0][0]
        x_l = left_eye[2][0]
        eye_dist = abs(x_r - x_l) / x_max


    y_nose1 = nose_landmarks[0][1]
    y_nose2 = nose_width_landmarks[-1][1]
    x_nose1 = nose_width_landmarks[0][0]
    x_nose2 = nose_width_landmarks[-1][0]

    nose_length = abs(y_nose2 - y_nose1) / y_max
    nose_width = abs(x_nose2 - x_nose1) / x_max

    return pd.Series({'Actor': row['Actor'],'Eye Distance': eye_dist, 'Nose Length': nose_length, 'Nose Width': nose_width})

In [ ]:
#Apply the face_proportions function to all actors 
facial_proportions = face_encodings.apply(face_proportions, axis=1)

>**Features visualisation** Now that we have obtained the features (proportions of different disctinctives facial features, such as nose length, distance between eyes, etc..)

In [ ]:
feature_columns = facial_proportions[['Eye Distance', 'Nose Length', 'Nose Width']]

# Create the pairplot
sns.pairplot(feature_columns, kind='scatter', diag_kind='hist', corner=True)
plt.suptitle('Feature visualisation', y=1.02, size=16)

plt.show()

> We observe that the nose length is normally distributed, whereas the eye distance and nose width are skewed to the right and left, respectively.  

In [ ]:
correlation_matrix = feature_columns.corr()

mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))

plt.figure(figsize=(8, 6))

sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', cbar=False, mask=mask)
plt.title('Correlation Matrix (Lower Triangular)')
plt.show()

The box plots allow us to visualise outliers . Disgarding feature outliers can improve model accuracy

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(data=feature_columns, palette='Set2')
plt.title('Box Plots for Facial Proportions')
plt.ylabel('Facial Proportions {%}')
plt.legend()
plt.show()

In [ ]:
column_summaries = pd.DataFrame(facial_proportions.describe())
display(column_summaries)